# Feature Set Generator :: ECDF

In [22]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import skew, kurtosis, linregress
from matplotlib import pyplot as plt
import multiprocessing
from multiprocessing import Pool
import os
# import math

In [25]:
# Path settings

# path_var = 1 

# if (path_var == 1):
#     filepath = '/home/docboy/Desktop/Option_Pricing/datasets/'

# else:
#     filepath = '/home/sharan/Desktop/Option_Pricing/datasets/'


filepath = '/home/docboy/Desktop/SBIN/'


In [26]:

# clean = pd.read_csv(filepath + 'ttm_cleaned_BANKNIFTY_2014_2018_raw.csv')
# clean = clean.dropna()
# filt = pd.read_csv(filepath + 'ttm_filter_param_0.04_BANKNIFTY_2015_2018.csv')
# ts_data = pd.read_csv(filepath + 'INDEXVALUE_BANKNIFTY_2014_2018_combined_raw.csv')


# For NIFTY50

clean = pd.read_csv(filepath + 'ttm_filter_param_0.04_SBIN_2014_2018.csv')
clean = clean.dropna()
filt = pd.read_csv(filepath + 'ttm_filter_param_0.04_SBIN_2015_2018.csv')
ts_data = pd.read_csv(filepath + 'INDEXVALUE_SBIN_2015_2018_combined.csv')


In [27]:
#print(clean)
filt['Date'] = pd.to_datetime(filt['Date'])
filt = filt.sort_values(by='Date')
filt.reset_index(inplace = True, drop = True)
clean['Date'] = pd.to_datetime(clean['Date'])
clean['Expiry'] = pd.to_datetime(clean['Expiry'])
clean = clean.sort_values(by='Date')
clean.reset_index(inplace = True, drop = False)
ts_data['Date'] = pd.to_datetime(ts_data['Date'])  
ts_data = ts_data.sort_values(by='Date')
ts_data.reset_index(inplace = True, drop = False)

In [28]:
clean

,index,Symbol,Date,Expiry,Option Type,Strike Price,Open,High,Low,Close,LTP,Settle Price,No. of contracts,Turnover in Lacs,Open Int,Change in OI,Underlying Value,ttm
0,0,SBIN,2014-10-07,2014-10-30,CE,2450,50.25,59.35,35.15,37.25,38.00,37.25,3069,9577.09,162125,51875,2366.40,23
1,1,SBIN,2014-10-07,2014-10-30,CE,2300,150.25,150.25,111.00,114.70,114.40,114.70,28,84.70,7000,625,2366.40,23
2,2,SBIN,2014-10-07,2014-10-30,CE,2350,107.25,107.25,78.00,81.20,85.50,81.20,47,143.04,10750,2875,2366.40,23
3,3,SBIN,2014-10-07,2014-10-30,CE,2400,75.00,85.15,52.95,55.05,57.30,55.05,1981,6099.72,139000,57875,2366.40,23
4,4,SBIN,2014-10-07,2014-11-27,CE,2400,100.00,100.00,100.00,100.00,100.00,100.00,1,3.13,250,125,2366.40,51
5,5,SBIN,2014-10-08,2014-10-30,CE,2400,51.80,82.00,51.40,78.85,79.80,78.85,2839,8767.59,125000,-14000,2411.25,22
6,6,SBIN,2014-10-08,2014-10-30,CE,2450,34.20,56.35,34.15,53.90,55.05,53.90,3226,10075.33,149875,-12250,2411.25,22
7,7,SBIN,2014-10-08,2014-10-30,CE,2500,22.25,37.00,21.95,35.35,36.50,35.35,4817,15245.21,412250,9375,2411.25,22
8,8,SBIN,2014-10-08,2014-10-30,CE,2350,78.35,114.00,78.35,111.85,111.00,111.85,141,431.34,12875,2125,2411.25,22
9,9,SBIN,2014-10-08,2014-11-27,CE,2500,74.10,74.10,70.00,71.55,71.55,71.55,5,16.07,1375,250,2411.25,50


### Option Data Functions

In [29]:
def ret(data, date, mode='s'):
    '''
    This function takes in 2 primary params. The function outputs a vector of simple return data
    corressponding to previous 20 days raw ts data.
    The function can also output the log returns on changing the value of the mode param to 'log'.

    ## Modification :: The function can also output just the raw rows if mode param is set to 'e'

    # Demo func
    ret(df, '31-Jan-2019')
    ret(df, '31-Jan-2019', 'log')
    ret(df, '31-Jan-2019', 'e')

    '''
#   print(data.loc[k,:])
    if (mode == 's'): # defualt Func logic
        val = pd.DataFrame()
        val.insert(0, "Return", 0)
        temp=((data['Close']-data['Close'].shift(1))/(data['Close'].shift(1)))*100
        val['Return']=temp[1:]

    if (mode == 'log'): # Func logic if log returns is desired
        val = pd.DataFrame()
        val.insert(0, "Log Return", 0)
        temp = np.log(data['Close'])-np.log(data['Close'].shift(1))
        val['Log Return']=temp[1:]

    if (mode == 'e'): # Func logic to extract the raw data
        k = data.index[data['Date']==date][0]
        val = pd.DataFrame()
        val.insert(0, "Close", 0)
        val.insert(0, "High", 0)
        val.insert(0, "Low", 0)
        val.insert(0, "Date", 0)
        val["Close"] = data.loc[k-19:k, "Close"]
        val['High'] = data.loc[k-19:k, "High"]
        val['Low'] = data.loc[k-19:k, "Low"]
        val['Open'] = data.loc[k-19:k, "Open"]
        val['Date'] = data.loc[k-19:k, "Date"]
    val.reset_index(inplace=True, drop=True)
    return val

### Feature Generator

In [30]:
def gen(ts_data, ttm, atmDate, atmError, strike,spot, opPrice, percentage):
    
    log_return = ret(ts_data, atmDate, mode = 'log')
    
    log_return = log_return.loc[:,'Log Return'].tolist()
        
    log_return.sort()
    
    return [atmDate] + log_return + [ttm, atmError, strike,spot, opPrice, percentage]

In [31]:
flist=[]
for i in range(1,20):
    fname='f'+ str(i)
    flist.append(fname)
col_name = ['date'] + flist + ['ttm','atmError','strike','spot','opPrice','percentage']

In [32]:
len(filt)

6806

In [33]:
filt.reset_index(drop=True,inplace=True)
# filt
clean.reset_index(drop=True,inplace=True)

In [34]:
# clean.loc[clean["Strike Price"] == 8600.00]

In [35]:
count = 0
# percentage = []

feat = []
params=[]
for index,row in filt.iterrows():
    ttm = row['ttm']
    strike=row['Strike Price']
    spot=row['Underlying Value']
    atmDate=row['Date']
    expDate=row['Expiry']
    close = row['Close']
    opPrice = close
    opsTS = clean.loc[np.logical_and(clean['Strike Price'] == strike, clean['Expiry'] == expDate)]
    opsTS=opsTS.drop("index",1)
    opsTS.reset_index(inplace=True, drop=True)
#     print(atmDate)
    ind = opsTS.index[opsTS['Date']==atmDate]#Gives Index of that day itself
#     print(ind)
#     print(opsTS)
    ind=ind[0]
    if (ind > 20):
        window = 20
    elif (ind < 1):
        continue        
    else:
        window = ind
        
#     if((spot-strike) >= opPrice):
#         continue
    
    percentage = (close/strike)*100
    
    ts = ret(ts_data, atmDate, mode = 'e')
    
    atmError=(spot/strike)*100
    params.append((ts,ttm,atmDate,atmError,strike,spot,opPrice,percentage))
#     break
    
pool = Pool()
for res in pool.starmap(gen,params):    
    feat.append(res)
    
pool.close()


In [36]:
results=pd.DataFrame.from_records(feat, columns=col_name)
results

,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f16,f17,f18,f19,ttm,atmError,strike,spot,opPrice,percentage
0,2015-01-01,28.000000,98.125000,320.000000,314.000000,9.000000,2.812500,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01,28.000000,99.682540,315.000000,314.000000,11.250000,3.571429,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-01,28.000000,96.615385,325.000000,314.000000,7.050000,2.169231,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-01,28.000000,102.950820,305.000000,314.000000,17.600000,5.770492,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-01,28.000000,101.290323,310.000000,314.000000,14.000000,4.516129,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015-01-01,56.000000,101.290323,310.000000,314.000000,20.400000,6.580645,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015-01-01,56.000000,99.682540,315.000000,314.000000,17.500000,5.555556,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2015-01-01,56.000000,98.125000,320.000000,314.000000,15.300000,4.781250,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015-01-02,0.003973,55.000000,100.079365,315.000000,315.250000,18.800000,5.968254,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015-01-02,0.003973,27.000000,103.360656,305.000000,315.250000,17.600000,5.770492,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
results = results.dropna()

In [38]:
print(len(results))

6067


In [39]:
# results.to_csv(filepath + 'BANKNIFTY_feature_set_ECDF_STEP_1.csv', index = False)

filepath = '/home/docboy/Desktop/Option_Pricing/datasets/'

results.to_csv(filepath + 'SBIN_feature_set_ECDF_STEP_1.csv', index = False)





In [17]:
# results.hist(column='ttm', bins =44)